In [3]:
import os
import json
import csv
import pandas as pd
import sys
from py2neo import Graph, Node
from collections import defaultdict
import re

In [8]:
import jieba.posseg as pseg

# 定义文本
text = '像这样的话 一个吊杆上的磁力计、宇宙射线计数器、一个辐射计。另有一台光电摄像机。'

# 分词并进行词性标注
words = pseg.cut(text)

# 提取文本中的实体
entities = []
for word, flag in words:
    if flag == 'n' or flag == 'nr' or flag == 'ns' or flag == 'nt' or flag == 'nz':
        entities.append(word)

# 输出实体列表
print(entities)


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\11766\AppData\Local\Temp\jieba.cache
Loading model cost 0.408 seconds.
Prefix dict has been built successfully.


['吊杆', '磁力计', '计数器', '辐射计', '摄像机']


In [4]:
import re

sentence = '这是一句话，（包含中文标点符号）。分割后应该得到三个句子！'

# 定义正则表达式模式，匹配包含中文标点符号的句子
pattern = r'([^。！？；，、‘’（）：……！？]*)(?:[。！？；，、‘’（）：……！？])?'

# 使用正则表达式模块的findall方法，查找所有匹配的句子
sentences = re.findall(pattern, sentence)

# 打印分割后的句子列表
print(sentences)

def is_contains_chinese(strs):
    for _char in strs:
        if '\u4e00' <= _char <= '\u9fa5':
            return True
    return False

['这是一句话', '', '包含中文标点符号', '', '分割后应该得到三个句子', '']


In [5]:
def read_node(header, relation, type, name, path, id): # 参数为 实体属性列表、关系列表、 本体类型 和 中文名称
    df = pd.read_excel(path, header=id)
    entities = {}
    relationships = defaultdict(list)
    for index, row in df.iterrows():
        if index == 0:
            continue
        # name 里面有中文、英文名称，应该分割出来
        allname = re.findall(pattern, row[name])
        try:
            entity = {'type': type}
            for val in allname:
                # print(val)
                if val == '':
                    continue
                if is_contains_chinese(val):
                    entity['name'] = val
                else:
                    entity['EnglishName'] = val
        except Exception as e:
            print(e)
        if 'name' not in entity:
            entity['name'] = entity['EnglishName']
        
        entities[entity['name']] = entity  
        ######################

        for key in header:
            key = key.strip()
            try:
                if row[key] == '——' or not row[key] or row[key] == '无':
                    continue
                entity[key] = row[key]
            except Exception as e:
                print(e)
       
        
        for key in relation:
            key = key.strip()
            try:
                if row[key] == '——' or not row[key]:
                    continue
                allkeys = re.findall(pattern, row[key])
                for x in allkeys:
                    if x == '':
                        continue
                    relationship = [entity['name'],entity['type'], x, key]
                    relationships[entity['type'] + '-->' + key].append(relationship)
            except Exception as e:
                print(e) 
                pass
    for name_t, entity in entities.items():
        print(name_t, entity)
    print('-----' * 10)
    for y in relationships:
        print(relationships[y])

In [243]:
question_path = "../data/【20230306V3】深空探测领域知识采集表格—1-科学问题.xlsx"
header = ['科学问题描述', '科学问题来源', '探测要素', '分析手段', '探测进展情况', '应用目标']
relation = ['上位科学问题', '探测对象', '探测载荷']
read_node(header, relation, '科学问题', '科学问题', question_path, id=1)

起源 {'type': '科学问题', 'name': '起源', '科学问题描述': '太阳系的起源', '科学问题来源': '2022年《起源、世界和生命：行星科学和天体生物学十年战略2023—2032》'}
世界和进程 {'type': '科学问题', 'name': '世界和进程', '科学问题描述': '太阳系世界和进程', '科学问题来源': '2022年《起源、世界和生命：行星科学和天体生物学十年战略2023—2032》'}
生命和宜居性 {'type': '科学问题', 'name': '生命和宜居性', '科学问题描述': '生命和宜居性', '科学问题来源': '2022年《起源、世界和生命：行星科学和天体生物学十年战略2023—2032》'}
综合 {'type': '科学问题', 'name': '综合', '科学问题描述': '系外行星，贯穿起源、世界和进程、生命和宜居性', '科学问题来源': '2022年《起源、世界和生命：行星科学和天体生物学十年战略2023—2032》'}
原行星盘的演化 {'type': '科学问题', 'name': '原行星盘的演化', '科学问题描述': '太阳系的初始条件是什么？哪些过程导致了行星构造块的产生，这些物质的性质和演化是什么？', '科学问题来源': '2022年《起源、世界和生命：行星科学和天体生物学十年战略2023—2032》'}
外太阳系的吸积 {'type': '科学问题', 'name': '外太阳系的吸积', '科学问题描述': '巨行星及其卫星系统是何时、以何种方式起源的，其轨道是否在历史早期阶段发生了迁移？矮行星和在巨行星之外运行的彗星是何时、以何种方式形成的，又是如何受到太阳系早期演化的影响？', '科学问题来源': '2022年《起源、世界和生命：行星科学和天体生物学十年战略2023—2032》'}
地球和内太阳系天体的起源 {'type': '科学问题', 'name': '地球和内太阳系天体的起源', '科学问题描述': '类地行星、卫星和小行星是何时、以何种方式吸积的，哪些过程决定了类地行星、卫星和小行星的初始性质？外太阳系物质在多大程度上被纳入其中？', '科学问题来源': '2022年《起源、世界和生命：行星科学和天体生物学十年战略2023—2032》'}
撞击和动

In [244]:
object_path = "../data/【20230306V3】深空探测领域知识采集表格—2-探测对象.xlsx"
header = ['英文名称',  '天体类型', '生命潜力', '尺寸', '质量', '形状', '与太阳的距离', '公转轨道', '自转轨道', '是否有环', '天体起源与演化规律', '结构', \
          '磁层', '表面', '大气和气候']
relation = ['卫星']
read_node(header, relation, '探测对象', '中文名称', object_path, id=1)

火星 {'type': '探测对象', 'name': '火星', '英文名称': 'Mars；Red Planet', '天体类型': '类地行星', '生命潜力': '科学家预计不会在火星上发现目前正在蓬勃发展的生物。不过，科学家正在寻找很久以前就存在的生命迹象，当时火星更加温软，覆盖着水。', '尺寸': '半径为3390km', '质量': '6.4171×10^23 kg', '形状': '球形', '与太阳的距离': '距离太阳平均距离约为2.28亿公里（km）或1.52天文单位（AU），远日点为1.67AU，近地点为1.38AU，偏心率为0.0934', '公转轨道': '周期约为687个地球日，或者669.6个火星日（sol）', '自转轨道': '自转周期为24.6小时（火星日），自转轴相对于围绕太阳的轨道平面倾斜25°，由于火星公转轨道，季节并不均匀分布，北半球的春天（南半球的秋天）是最长的季节，为194个火星日，北半球的秋天（南半球的春天）最短，为142个火星日，北半球的冬季（南半球的夏季）为154个火星日，北半球的夏季（南半球的冬季）为178个火星日。', '天体起源与演化规律': '大约45亿年前，当太阳系进入目前的布局时，火星形成于引力吸引旋转的气体和尘埃进入。火星和其他类地行星一样，有一个中心核心、一个岩石地幔和一个坚固的地壳。', '结构': '火星中心有一个密集的核心，半径在1500km到2100km之间。由铁、镍和硫制成。围绕地核的是一层1240km到1880km厚的岩石地幔，在其上方是一层由铁、镁、铝、钙和钾组成的地壳，地壳的深度在10km到50km之间。', '磁层': '当前，火星没有全球磁场，但南半球的火星地壳区域被高度磁化，显示出40亿年前的磁场痕迹。', '表面': '表面颜色主要是棕色、金色和棕褐色等，火星看起来微红的原因是由于岩石、表土和火星尘埃中的铁被氧化。火星表面积几乎与地球陆地面积相同。火星拥有一个名为“水手谷”的大峡谷，超过4800km。火星峡谷最宽为320km，最深7km。火星似乎曾经有过一段充满水的过去，有古老的河谷网络、三角洲和湖床，以及表面上只能在液态水中形成的岩石和矿物。一些特征表明，火星在大约35亿年前 经历了巨大的洪水。当前火星上有水，但火星大气层太稀薄 ，液态水无法在火星表面长期存在，火星上

In [6]:
task_path = "../data/【20230306V3】深空探测领域知识采集表格—3-探测任务.xlsx"
header = ['发射时间', '任务简述', '任务科学目标 ', '对应科学主题 ', '对应学科 ', '首席科学家 ', '主要探测方式', '探测方式', '分析手段', '工程目标', '任务关键技术',\
           '研制单位', '任务负责人', '任务经费','成本控制措施', '轨道设计', '运载火箭', '发射质量/kg', '发射场', '任务结果', '失败类型和具体故障情况', '当前状态', '标志性事件',\
           '研究成果','任务应用目标', '任务应用方案', '任务相关军事因素', '任务相关经济因素', '任务相关政治因素', '任务来源']
relation = ['所属国家', '所属探测计划', '合作国家', '主要探测对象', '其他探测对象', '解答的科学问题 ', '有效载荷']

df = pd.read_excel(task_path, header=2)
# for idx, row in df.iterrows():
#     if idx == 0:
#         continue
#     print(row)
#     break
read_node(header, relation, '探测任务', '任务名称',task_path, id=2)

expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
火星-1960A {'type': '探测任务', 'name': '火星-1960A', 'EnglishName': 'Mars 1960A', '发射时间': datetime.datetime(1960, 10, 10, 0, 0), '任务简述': '任务目标是调查地球和火星之间的行星际空间，研究火星并从飞越轨道返回表面图像，研究长时间太空飞行对机载仪器的影响，提供远距离无线电通信。', '任务科学目标': '研究火星附近的宇宙射线、磁场、微陨石和大气环境、有机物构成。', '对应科学主题': '世界和进程', '对应学科': '行星科学', '主要探测方式': '飞越', '探测方式': '飞越', '分析手段': '遥感分析', '工程目标': '火星飞越', '任务关键技术': '深空飞行技术、深空通信技术', '研制单位': '试验设计一局(OKB-1)', '任务负责人': 'V.G.Perminov（首席设计师）', '轨道设计': '火星飞越轨道', '运载火箭': '8K78，即闪电号（ Molniya）', '发射质量/kg': 650, '发射场': '拜科努尔发射场 ', '任务结果': '失败', '失败类型和具体故障情况': '发射失败：运载火箭第三级故障，未能进入地球停泊轨道', '当前状态': '失效', '标志性事件': '苏联第一个火星探测器。', '任务相关军事因素': '美苏争霸', '任务相关政治因素': '美苏争霸', '任务来源': '为实现在航天领域的领先，苏联率先开始探测火星，实施了“火星”（Mars）计划'}
火星-1960B {'type': '探测任务', 'name': '火星-1960B', 'EnglishName': 'Mars 1960B', '发射时间': datetime.datetime(1960, 10, 14, 0, 0), '任务简述': '目的是研究火星附近的宇宙射线、磁场、微陨石和大气环境、有机物构成', '任务科学目标': '研究火星附近的宇宙射线、磁场、微

In [246]:
detector_path = "../data/【20230306V3】深空探测领域知识采集表格—4-探测器.xlsx"
df = pd.read_excel(detector_path, header=2)
header = ['发射时间', '探测要素', '探测器科学探测目标', '对应学科', '探测器探测方式', '探测器分析手段', '探测器关键技术 ', '探测器主要参数', '探测器功能指标', '探测器性能指标',\
          '探测器寿命', '探测器研制单位', '探测器负责人', '探测器当前状态', '探测器应用目标', '探测器应用方案']
relation = ['所属任务','探测对象', '探测器有效载荷', '探测器研制单位所属国家', ]
read_node(header, relation, '探测器', '探测器名称',detector_path, id=2)
# for idx, row in df.iterrows():
#     if idx == 0:
#         continue
#     print(row)
#     break

火星-1960A {'type': '探测器', 'name': '火星-1960A', 'EnglishName': 'Mars 1960A', '发射时间': datetime.datetime(1960, 10, 10, 0, 0), '探测要素': '火星附近的宇宙射线、磁场、微陨石和大气环境、有机物构成', '探测器科学探测目标': '研究火星附近的宇宙射线、磁场、微陨石和大气环境、有机物构成。', '对应学科': '行星科学', '探测器探测方式': '飞越', '探测器分析手段': '遥感分析', '探测器关键技术': '深空飞行技术、深空通信技术', '探测器主要参数': '高约2米的圆柱体，带有两个太阳能电池板，2.33米高的高增益网状天线，1个长天线臂，质量约650kg。', '探测器功能指标': '携带10kg科学载荷', '探测器研制单位': '试验设计一局(OKB-1)', '探测器负责人': 'V.G.Perminov（首席设计师）', '探测器当前状态': '失效'}
火星-1960B {'type': '探测器', 'name': '火星-1960B', 'EnglishName': 'Mars 1960B', '发射时间': datetime.datetime(1960, 10, 14, 0, 0), '探测要素': '火星附近的宇宙射线、磁场、微陨石和大气环境、有机物构成', '探测器科学探测目标': '研究火星附近的宇宙射线、磁场、微陨石和大气环境、有机物构成。', '对应学科': '行星科学', '探测器探测方式': '飞越', '探测器分析手段': '遥感分析', '探测器关键技术': '深空飞行技术、深空通信技术', '探测器主要参数': '高约2米的圆柱体，带有两个太阳能电池板，2.33米高的高增益网状天线，1个长天线臂，质量约650kg。', '探测器功能指标': '携带10kg科学载荷', '探测器研制单位': '试验设计一局(OKB-1)', '探测器负责人': 'V.G.Perminov（首席设计师）', '探测器当前状态': '失效'}
火星-1962A {'type': '探测器', 'name': '火星-1962A', 'EnglishName': 'Mars 1962A', '发射时间': dateti

In [247]:
load_path = "../data/【20230306V3】深空探测领域知识采集表格—5-探测载荷.xlsx"
header = ['载荷类型', '载荷当前状态 ','发射时间', '探测要素', '载荷科学探测目标', '对应学科', '载荷分析手段', '载荷探测波段', '载荷关键技术', '载荷主要参数',\
          '载荷功能指标', '载荷性能指标', '载荷寿命','载荷研制单位', '载荷负责人', '载荷应用目标', '载荷应用方案']
relation = ['所属探测器', '探测主要对象', '载荷研制单位所属国家']

df = pd.read_excel(load_path, header=2)
# for idx, row in df.iterrows():
#     if idx == 0:
#         continue
#     print(row)
#     break
read_node(header, relation, '探测载荷', '载荷名称', load_path, id=2)

光谱反射计 {'type': '探测载荷', 'name': '光谱反射计', '载荷当前状态': '已应用', '发射时间': datetime.datetime(1962, 11, 1, 0, 0), '探测要素': '火星大气', '载荷分析手段': '遥感分析'}
辐射探测器 {'type': '探测载荷', 'name': '辐射探测器', '载荷类型': '辐射测量仪', '载荷当前状态': '已应用', '发射时间': datetime.datetime(1969, 4, 2, 0, 0), '探测要素': '火星表面', '载荷分析手段': '遥感分析'}
宇宙射线探测器 {'type': '探测载荷', 'name': '宇宙射线探测器', '载荷类型': '辐射测量仪', '载荷当前状态': '已应用', '发射时间': datetime.datetime(1971, 5, 10, 0, 0), '探测要素': '火星表面', '载荷分析手段': '遥感分析'}
光电摄像机 {'type': '探测载荷', 'name': '光电摄像机', '载荷当前状态': '已应用', '发射时间': datetime.datetime(1960, 10, 14, 0, 0), '探测要素': '火星表面', '载荷科学探测目标': '获取火星表面图像', '对应学科': '行星科学', '载荷分析手段': '遥感分析'}
微陨石探测器 {'type': '探测载荷', 'name': '微陨石探测器', '载荷类型': '压电传感器', '载荷当前状态': '已应用', '发射时间': datetime.datetime(1964, 11, 30, 0, 0), '探测要素': '宇宙辐射中的流星粒子和质子', '载荷科学探测目标': '研究宇宙辐射中的流星粒子和质子', '对应学科': '行星科学：场和粒子', '载荷分析手段': '就位分析', '载荷性能指标': '可探测区域为 1.5 平方米。传感器探测质量为 0.1 微克或更大的粒子。', '载荷研制单位': 'Russian Academy of Sciences', '载荷负责人': 'T. N. Nazarova'}
成像系统 {'type': '探测载荷', 'name': '成像系统',

In [248]:
technology_path = "../data/【20230306V3】深空探测领域知识采集表格—6-探测技术.xlsx"
header = ['探测技术一级分类', '探测技术二级分类', '探测技术三级分类', '探测技术来源', '技术描述', '技术挑战']
relation = ['探测对象']
read_node(header, relation, '探测技术', '探测技术名称',technology_path, id=1)

可见光/近红外焦平面阵列 {'type': '探测技术', 'name': '可见光/近红外焦平面阵列', '探测技术一级分类': '载荷技术', '探测技术二级分类': '遥感仪器和传感器', '探测技术三级分类': '探测器和焦平面', '探测技术来源': '《2015NASA技术路线图》', '技术描述': '大幅面可见光/近红外（IR）探测器阵列', '技术挑战': '在实现低噪声的同时降低像素尺寸和密度'}
红外焦平面 {'type': '探测技术', 'name': '红外焦平面', '探测技术一级分类': '载荷技术', '探测技术二级分类': '遥感仪器和传感器', '探测技术三级分类': '探测器和焦平面', '探测技术来源': '《2015NASA技术路线图》', '技术描述': '用于成像仪、光谱仪和成像光谱仪的红外交平面', '技术挑战': '探测器材料，探测器制造，数字读出电路，高温工作，低暗电流'}
滤波器成像仪 {'type': '探测技术', 'name': '滤波器成像仪', 'EnglishName': '2D', '探测技术一级分类': '载荷技术', '探测技术二级分类': '遥感仪器和传感器', '探测技术三级分类': '探测器和焦平面', '探测技术来源': '《2015NASA技术路线图》', '技术描述': '用于二维（2D）成像仪的光谱分辨率滤波器', '技术挑战': '开发这类滤波器需要超净的专门设施，因为其极易受到污染'}
亚毫米波阵列光谱仪 {'type': '探测技术', 'name': '亚毫米波阵列光谱仪', '探测技术一级分类': '载荷技术', '探测技术二级分类': '遥感仪器和传感器', '探测技术三级分类': '探测器和焦平面', '探测技术来源': '《2015NASA技术路线图》', '技术描述': '高分辨率亚毫米波多像素光谱仪', '技术挑战': '此前尚未有远红外波段外差式阵列技术'}
红外探测器 {'type': '探测技术', 'name': '红外探测器', '探测技术一级分类': '载荷技术', '探测技术二级分类': '遥感仪器和传感器', '探测技术三级分类': '探测器和焦平面', '探测技术来源': '《2015NASA技术路线图》', '技术描述': '可承受空间辐射

In [249]:
class DeepspaceGraphe:
    # 后续得设计一个字典来映射 同一个表格中的同本体之间的关系
    def __init__(self, g, question_path, object_path, task_path, detector_path, load_path, technology_path) -> None:
        self.g = g
        cur_dir = '/'.join(os.path.abspath(__file__).split('/')[:-1])
        self.question_path = os.path.join(cur_dir, question_path)
        self.object_path = os.path.join(cur_dir, object_path)
        self.task_path = os.path.join(cur_dir, task_path)
        self.detector_path = os.path.join(cur_dir, detector_path)
        self.load_path = os.path.join(cur_dir, load_path)
        self.technology_path = os.path.join(cur_dir, technology_path)
        self.entitise = {} # 存储节点
        self.relationships = defaultdict(list) # 存储关系
        
    def read_node(self,header, relation, type, name, path, id): # 参数为 实体属性列表、关系列表、 本体类型 和 中文名称
        df = pd.read_excel(path, header=id)
        for index, row in df.iterrows():
            if index == 0:
                continue
            # name 里面有中文、英文名称，应该分割出来
            allname = re.findall(pattern, row[name])
            try:
                entity = {'type': type}
                for val in allname:
                    # print(val)
                    if val == '':
                        continue
                    if is_contains_chinese(val):
                        entity['name'] = val
                    else:
                        entity['EnglishName'] = val
            except Exception as e:
                print(e)
            if 'name' not in entity:
                entity['name'] = entity['EnglishName']
            self.entities[entity['name']] = entity 

            for key in header:
                key = key.strip()
                try:
                    if row[key] == '——' or not row[key] or row[key] == '无':
                        continue
                    entity[key] = row[key]
                except Exception as e:
                    print(e)
        
            
            for key in relation:
                key = key.strip()
                try:
                    if row[key] == '——' or not row[key]:
                        continue
                    allkeys = re.findall(pattern, row[key])
                    for x in allkeys:
                        if x == '':
                            continue
                        relationship = [entity['name'],entity['type'], x, key]
                        self.relationships[entity['type'] + '-->' + key].append(relationship)
                except Exception as e:
                    print(e) 
                    pass
        # for name_t, entity in entities.items():
        #     print(name_t, entity)
        # print('-----' * 10)
        # for y in relationships:
        #     print(relationships[y])
    
    def read_allnodes(self):
        
    


IndentationError: expected an indented block (1340472061.py, line 70)